In [2]:
# Import packages 
import pandas as pd
import numpy as np
# Set Working Directory
import os
# os.chdir(r'C:\Personal\IMM') # absolute path, using \ and r prefix
# wd = os.getcwd()
from itertools import chain
import timeit
from decimal import *
import traceback
from itertools import cycle
import itertools
import random
# set random seed to be used
seed = 12345
prng = np.random.RandomState(seed)

dirListing = 'c:\\personal\\IMM'

In [3]:
%%time
# batch in Rick's file that is the ALC for auto drive only and select a fraction due to memory issues.
# There are some zones with negative numbers. Just drop them for now
tour_df = pd.read_csv(r"C:\Personal\IMM\tours_out_with_start_time.gz", compression='gzip', sep=',')
tour_df = tour_df[['tour_id', 'start_time']]
tour_df.rename(columns={'start_time': 'tour_starttime'}, inplace=True)

temp_df = pd.read_csv(r"c:/personal/imm/trips_out_autoDriver.gz", compression='gzip', sep=',')
temp_df = temp_df.loc[(temp_df['origin_zone'] >0) &(temp_df['destination_zone'] >0)]

Wall time: 59 s


In [4]:
temp_df = pd.merge(temp_df, tour_df, on='tour_id')

In [5]:
# sorting order for sure if you want TOD to work, period.
temp_df.sort_values(['hhid', 'pid','tour_id', 'tour_seq'], inplace=True)

In [6]:
%%time
# batch in SOV non toll times for Treso and then melt it and rename columns
ttime = pd.read_csv(r"c:/personal/imm/ff_ttimes.gz", compression='gzip', sep=',')
ttime.rename(columns={'0187474001524615141/': 'origin_zone'}, inplace = True)
ttime = pd.melt(ttime, id_vars=["origin_zone"]).dropna()
ttime.rename(columns={'variable':'destination_zone', 'value':'ttime_mins'}, inplace = True)

Wall time: 34.5 s


In [7]:
%%time
# set dictionary of Dtypes to reduce size requirement of travel time dataframe
dtype_ttime = {}

# dictionary for storing hhold column dtypes
dtype_ttime = {'origin_zone':'int16',
              'destination_zone':'int16',
              'ttime_mins':'float64'
              }

for key, value in dtype_ttime.items():
    ttime[key] = ttime[key].astype(value)

Wall time: 6.3 s


In [8]:
%%time
# Transfer O-D level auto travel times
temp_df = pd.merge(temp_df, ttime, how = 'left', on = ['origin_zone', 'destination_zone'])

Wall time: 19.1 s


In [10]:
temp_df.head()

,origin_activity,destination_activity,trip_purpose,trip_id,tour_id,tour_seq,is_virtual,hhid,pid,origin_zone,destination_zone,trip_km,primary_mode,tour_starttime,ttime_mins
0,work,home,HBW,15269149,13551179,1,False,1,0,1516,1001,-1.00000,Auto Driver,8.00,25.464621
1,home,shop,HBM,15269150,13551180,0,False,2,0,1001,1014,4.03487,Auto Driver,9.00,4.469987
2,shop,home,HBM,15269151,13551180,1,False,2,0,1014,1001,4.03487,Auto Driver,9.00,4.469987
3,home,work,HBW,15269152,13551181,0,False,2,0,1001,2520,-1.00000,Auto Driver,13.83,29.577557
4,work,home,HBW,15269153,13551181,1,False,2,0,2520,1001,-1.00000,Auto Driver,13.83,30.324661


In [9]:
temp_df.loc[temp_df['trip_km'] == -1].shape[0]

6279620

In [10]:
# Add in direction. It is outbound if the origin_activity has 'home' in it and inbound if 'home' is in the destination_activity.
# Further, because the NHBO is directionless, at least w.r.t to the home, they have been all assigned inbound as
# the distributions in the start time file are the same for inbound and outbound for these the NHBO trip purpose. 
# The NHBW is treated the same as the WBO trip purpose from the GGHM and if there is 'work' in the origin_activity it gets
# an outbound and inbound if 'work' is in the destination_activity. 
# Finally, Business to Business is assigned a direction randomly.

In [12]:
%%time
# Home in one trip end
temp_df['direction'] = np.nan
temp_df.loc[(temp_df['origin_activity'] == 'home'), 'direction'] = 'outbound' 
temp_df.loc[(temp_df['direction'].isnull())&(temp_df['destination_activity'] == 'home'), 'direction'] = 'inbound' 

# NHBO
temp_df.loc[(temp_df['trip_purpose'] == 'NHBO'), 'direction'] = 'inbound'

# NHBW
temp_df.loc[(temp_df['direction'].isnull())&(temp_df['origin_activity'] == 'work'), 'direction'] = 'outbound' 
temp_df.loc[(temp_df['direction'].isnull())&(temp_df['destination_activity'] == 'work'), 'direction'] = 'inbound' 

#Business, randomly select 50% of the records that were yet NAN and give them outbound with the
#rest as inbound
random_direction = temp_df[temp_df['direction'].isnull()].sample(frac=0.5, random_state = seed)
temp_df.loc[(temp_df.index.isin(random_direction.index)), 'direction'] = 'outbound'
temp_df.loc[(temp_df['direction'].isnull()), 'direction'] = 'inbound'


Wall time: 23.8 s


In [13]:
temp_df.isnull().values.any()

False

# Now Create Start Times

### Batch in the Time Distributions that Nem created.

### Second, I have split NHB into inbound and outbound by keeping their values their same

In [14]:
time_dist = pd.read_csv(r"c:/personal/imm/start_time_distribution_treso.csv")

## Get the dataframe slice that corresponds to the purpose and direction. This is the old function, so IGNORE

# Older Time Allocation Code with Constraints...finalized on Saturaday April 28, 2018
# But improved on April 29, 2018

%%time

results_frow = []
results1 = []


for name, group in temp_df1.groupby(['hhid', 'pid']):
 

    # get list of indices where the tour sequence is set to 0
    newtour_ind = np.where(group['tour_seq'] == 0)   
    newtour_ind = list(chain(*newtour_ind))
    
    # ACCOUNT FOR THE FACT THAT THERE ARE QUITE A FEW TOURS THAT BELONG TO A PERSON THAT DO NOT HAVE A ZERO IN TOUR_SEQ 
    if len(newtour_ind) == 0:
        start_val = group.at[group.index[0], 'tour_starttime']
        results_frow.append([start_val])
        results_frow.append([0])
        
        continue
#     print("The tour index is %s" %newtour_ind)
    
    tour_num = 0
    total_tour_number = len(group['tour_id'].unique())
#     print("the number of tours are %s" % total_tour_number)
    
    
    for tour_name, tour_group in group.groupby('tour_id'):
        
        tour_num = tour_num + 1
#         print("current tour num is: %s" %tour_num)

#         print("Current tour name is: %s" % tour_name)

        # If there was only one tour made by the person than this scalar would be zero and it would skip the constraints
        # calculation
        len_newtour_ind = total_tour_number - tour_num 
#         print("Current tour number is: %s" %len_newtour_ind)
        
        # CONSTRAINTS CALCULATION. 
        # Here a preceeeding tour of a person has to keep in mind that it does not violate the start time of the
        # subsequent tour made by the same person
        if (len_newtour_ind) > 0:
#             print("do while loop")
            for i in range(0, tour_group.shape[0]):

                if i == 0:
#                     print("first row of while loop")
                    # the trip start time is the same as tour start time
                    start_time_first = tour_group.at[tour_group.index[i], 'tour_starttime']
                    arrival_time_first = start_time_first + tour_group.at[tour_group.index[i], 'ttime_mins']/60
                    results_frow.append([start_time_first])
                    results_frow.append([arrival_time_first])
#                     print(results_frow)
                else:
#                     print("subs row of while loop")
                    
                    # ACCOUNT FOR THE FACT THAT SOMETIMES THE SUBSEQUENT TOUR THAT ACTS AS A CONSTRAINT HAS NO
                    # TRIP WHERE THE TOUR_SEQ IS ZERO. THIS IS BECAUSE IN A TRIP BASED MC MODEL ONE COULD HAVE EACH
                    # LEG OF THE TOUR WITH A DIFFERENT MODE.
                
                    try:
                        # get the position for the constraint and the constraint time.
                        constraint_ind = newtour_ind[tour_num]
                        constraint_time = group.at[group.index[constraint_ind], 'tour_starttime']
                    except :
                        constraint_time = 23.5
                        
                    # subset the time distribution dataframe
                    arrival_time_prev = results_frow[i-1]
                    time_dist1 = time_dist.loc[(time_dist['Time'] >= arrival_time_prev) & (time_dist['Time'] <= constraint_time)]

                    # get row purpose and direction to make unique column name for sampling from
                    row_purp = tour_group.at[tour_group.index[i], 'trip_purpose']
                    row_dir = tour_group.at[tour_group.index[i],'direction']
                    weight_column = row_purp + '_' + row_dir 
                    
                    # ACCOUNT FOR THE FACT THAT SOMETIMES THERE IS NO TIME DISTRIBUTION LEFT. JUST GRAB THE ARRIVAL TIME
                    # FROM THE PRECEEDING TRIP AS THE START TIME OF THE TRIP IN QUESTION
                    # sample a time and calculate a new arrival time as a result
                    if len(time_dist1)> 0:
                        
                        start_time = time_dist1.sample(n=1, weights= time_dist1[weight_column], replace=True, random_state=prng)
                        start_time = start_time[['Time']].values ###
                        start_time = start_time[0][0]
                    
                    else:
                        start_time = results_frow[-1]
                    
                    newarrival_time = start_time + tour_group.at[tour_group.index[i], 'ttime_mins']/60
                    results_frow.append([start_time])
#                     print("Start time of subsequent row is %s" %results_frow)
                    results_frow.append([newarrival_time])
#                     print("Arrival time of subsequent row is %s" %newarrival_time)

                
          
                
#                 tour_num += 1

        # If there is only one tour to begin with
        else:
            print("run single tour")
            for i in range(0, tour_group.shape[0]):
#                 print(tour_group.shape[0])
#                 print(i)
                
                if i == 0:
                  
                    # the trip start time is the same as tour start time
                    start_time_first = tour_group.at[tour_group.index[i], 'tour_starttime']
                    arrival_time_first = start_time_first + tour_group.at[tour_group.index[i], 'ttime_mins']/60
                    results_frow.append([start_time_first])
                    results_frow.append([arrival_time_first])
#                     print(results_frow)
                else:
                                    
#                     print("move to next row")
                    arrival_time_prev = results_frow[-1][0]
                    time_dist1 = time_dist.loc[time_dist['Time'] >= arrival_time_prev]
                 
#                     print(len(time_dist1))

                    # get row purpose and direction to make unique column name for sampling from
                    row_purp = tour_group.at[tour_group.index[i], 'trip_purpose']
                    row_dir = tour_group.at[tour_group.index[i],'direction']
                    weight_column = row_purp + '_' + row_dir 

                    # sample a time and calculate a new arrival time as a result
                    if len(time_dist1)> 0:
                        
                        start_time = time_dist1.sample(n=1, weights= time_dist1[weight_column], replace=True, random_state=prng)
                        start_time = start_time[['Time']].values ###
                        start_time = start_time[0][0]                        

                    else:
                        start_time = results_frow[-1][0]
                       
                    newarrival_time = start_time + tour_group.at[tour_group.index[i], 'ttime_mins']/60
                    results_frow.append([start_time])
#                   print("Start time of subsequent row is %s" %results_frow)
                    results_frow.append([newarrival_time])
#                   print("Arrival time of subsequent row is %s" %newarrival_time)



    

#         tour_num = tour_num + 1
    #         print(tour_num)



            


In [ ]:
temp_df1 = temp_df.loc[(temp_df['hhid'] > 0) & (temp_df['hhid'] <= 90) & (temp_df['trip_km'] > 0)]
print(temp_df1.shape[0])

# temp_df1 = temp_df.loc[temp_df['tour_id'] == 13558919]


# NEW ATTEMPT THAT IS DESIGNED TO MAKE SURE THAT THE RESULTS HAVE THE SAME LENGTH AS THE INPUT DATAFRAME

### Currently, take out all those trips that have a -1 in the Trip_Km column. This saves considerable amount of time

###  Check to resolve HOUSEHOLD ID == 75 for double squares

In [163]:
temp_df1 = temp_df.loc[temp_df['origin_zone'] != temp_df['destination_zone']]
temp_df1.shape[0]

16633112

In [139]:
temp_df2 = temp_df.loc[(temp_df['hhid'] > 150570) & (temp_df['hhid'] <= 151070)]

In [106]:
temp_df2.shape

(256, 16)

In [140]:
%%time

results_frow = []
results1 = []
LOG_EVERY_N = 1000

for name, group in temp_df1.groupby(['hhid', 'pid']):
    val = group.at[group.index[0], 'tour_id']
    if val % 10000 == 0:
        print(val)

    # get list of indices where the tour sequence is set to 0
    newtour_ind = np.where(group['tour_seq'] == 0)   
    newtour_ind = list(chain(*newtour_ind))
#     print("The length is: %s" %len(newtour_ind))
#     print("The indices of tours where a tour_seq is zero are %s" %newtour_ind)
    
    # ACCOUNT FOR THE FACT THAT THERE ARE QUITE A FEW TOURS THAT BELONG TO A PERSON THAT DO NOT HAVE A ZERO IN TOUR_SEQ.
    # IF A PERSON HAS MULTIPLE SUCH TOURS THE LOGIC BELOW WILL NOT ENSURE CONSISTENCY. THIS WAS DONE AS THE FINAL 
    # DATAFRAME WILL HOPEFULLY NOT HAVE SUCH AN ISSUE
    if len(newtour_ind) == 0:
        
        for i in range(0, group.shape[0]):
#             print("check", group.shape[0], i)
            
            if i == 0:
                
                start_time = group.at[group.index[i], 'tour_starttime']
                arrival_time = start_time + group.at[group.index[i], 'ttime_mins']/60
                results_frow.append(start_time)
                results_frow.append(arrival_time)
#                 print(results_frow)
            else:
                
                arrival_time_prev = results_frow[-1]
#                 print("Arrival time of previous trip is: %s" %arrival_time_prev)
                time_dist1 = time_dist.loc[time_dist['Time'] > arrival_time_prev]
#                 print("the length of the trip dist times df is: %s" %len(time_dist1))

                # get row purpose and direction to make unique column name for sampling from
                row_purp = group.at[group.index[i], 'trip_purpose']
                row_dir = group.at[group.index[i],'direction']
                weight_column = row_purp + '_' + row_dir 

                # sample a time and calculate a new arrival time as a result
                if len(time_dist1)> 0:

                    start_time = time_dist1.sample(n=1, weights= time_dist1[weight_column], replace=True, random_state=prng)
                    start_time = start_time[['Time']].values ###
                    start_time = start_time[0][0]                        

                else:
                    start_time = results_frow[-1]

                newarrival_time = start_time + group.at[group.index[i], 'ttime_mins']/60
                results_frow.append(start_time)
                results_frow.append(newarrival_time)
#                 print("Start time of this trip is: %s " %start_time)
#                 print("Arrival time of this trip is: %s " % newarrival_time)

        
        continue

    
    tour_num = 0
    total_tour_number = len(group['tour_id'].unique())
#     print("the number of tours are %s" % total_tour_number)
    
    
    for tour_name, tour_group in group.groupby('tour_id'):
        
        tour_num = tour_num + 1
#         print("current tour num is: %s" %tour_num)

        print("Current tour name is: %s" % tour_name)

        # If there was only one tour made by the person than this scalar would be zero and it would skip the constraints
        # calculation
        len_newtour_ind = total_tour_number - tour_num 
#         print("Current tour number is: %s" %len_newtour_ind)
        
        # CONSTRAINTS CALCULATION. 
        # Here a preceeeding tour of a person has to keep in mind that it does not violate the start time of the
        # subsequent tour made by the same person
        if (len_newtour_ind) > 0:
#             print("do while loop")
            for i in range(0, tour_group.shape[0]):

                if i == 0:
#                     print("first row of while loop")
                    # the trip start time is the same as tour start time
                    start_time_first = tour_group.at[tour_group.index[i], 'tour_starttime']
                    arrival_time_first = start_time_first + tour_group.at[tour_group.index[i], 'ttime_mins']/60
                    results_frow.append(start_time_first)
                    results_frow.append(arrival_time_first)
#                     print(results_frow)
                else:
                    print("This is the iterator %s" %i)
                    
                    # ACCOUNT FOR THE FACT THAT SOMETIMES THE SUBSEQUENT TOUR THAT ACTS AS A CONSTRAINT HAS NO
                    # TRIP WHERE THE TOUR_SEQ IS ZERO. THIS IS BECAUSE IN A TRIP BASED MC MODEL ONE COULD HAVE EACH
                    # LEG OF THE TOUR WITH A DIFFERENT MODE.
                
                    try:
                        # get the position for the constraint and the constraint time.
                        constraint_ind = newtour_ind[tour_num]
                        constraint_time = group.at[group.index[constraint_ind], 'tour_starttime']
                    except :
                        constraint_time = 23.5
                        
                    # subset the time distribution dataframe
                    print("This is frow %s" %results_frow )
                    arrival_time_prev = results_frow[-1]
                    print("Get the pevious arrival time %s " %arrival_time_prev)
                    time_dist1 = time_dist.loc[(time_dist['Time'] > arrival_time_prev) & (time_dist['Time'] < constraint_time)]

                    # get row purpose and direction to make unique column name for sampling from
                    row_purp = tour_group.at[tour_group.index[i], 'trip_purpose']
                    row_dir = tour_group.at[tour_group.index[i],'direction']
                    weight_column = row_purp + '_' + row_dir 
                    
                    # ACCOUNT FOR THE FACT THAT SOMETIMES THERE IS NO TIME DISTRIBUTION LEFT. JUST GRAB THE ARRIVAL TIME
                    # FROM THE PRECEEDING TRIP AS THE START TIME OF THE TRIP IN QUESTION
                    # sample a time and calculate a new arrival time as a result
                    print("The length of the TD matrix is %s" %len(time_dist1))
                    if len(time_dist1)> 0:
                        
                        start_time = time_dist1.sample(n=1, weights= time_dist1[weight_column], replace=True, random_state=prng)
                        start_time = start_time[['Time']].values ###
                        start_time = start_time[0][0]
                        print("This is %s" %start_time)
                    else:
                        print(results_frow)
                        start_time = results_frow[-1]
                        
                    
                    newarrival_time = start_time + tour_group.at[tour_group.index[i], 'ttime_mins']/60
                    results_frow.append(start_time)
#                     print("Start time of subsequent row is %s" %results_frow)
                    results_frow.append(newarrival_time)
#                     print("Arrival time of subsequent row is %s" %newarrival_time)

        # IF THERE IS ONLY ONE TOUR TO BEGIN WITH THEN THERE IS NO CONSTRAINT
        else:
#             print("run single tour")
            for i in range(0, tour_group.shape[0]):
#                 print(tour_group.shape[0])
#                 print(i)
                
                if i == 0:
                  
                    # the trip start time is the same as tour start time
                    start_time_first = tour_group.at[tour_group.index[i], 'tour_starttime']
                    arrival_time_first = start_time_first + tour_group.at[tour_group.index[i], 'ttime_mins']/60
                    results_frow.append(start_time_first)
                    results_frow.append(arrival_time_first)
#                     print(results_frow)
                else:
                                    
#                     print("move to next row")
                    arrival_time_prev = results_frow[-1]
                    time_dist1 = time_dist.loc[time_dist['Time'] > arrival_time_prev]
                 
#                     print(len(time_dist1))

                    # get row purpose and direction to make unique column name for sampling from
                    row_purp = tour_group.at[tour_group.index[i], 'trip_purpose']
                    row_dir = tour_group.at[tour_group.index[i],'direction']
                    weight_column = row_purp + '_' + row_dir 

                    # sample a time and calculate a new arrival time as a result
                    if len(time_dist1)> 0:
                        
                        start_time = time_dist1.sample(n=1, weights= time_dist1[weight_column], replace=True, random_state=prng)
                        start_time = start_time[['Time']].values ###
                        start_time = start_time[0][0]                        

                    else:
                        start_time = results_frow[-1]
                       
                    newarrival_time = start_time + tour_group.at[tour_group.index[i], 'ttime_mins']/60
                    results_frow.append(start_time)
#                   print("Start time of subsequent row is %s" %results_frow)
                    results_frow.append(newarrival_time)
#                   print("Arrival time of subsequent row is %s" %newarrival_time)


Current tour name is: 7485784
Current tour name is: 7514367
Current tour name is: 7460046
This is the iterator 1
This is frow [7.4199999999999999, 7.5596144390106197, 17.25, 18.04285297393799, 19.0, 19.801841862996419, 7.5, 7.949229558308919, 14.5, 15.193816630045573, 18.0, 18.684121640523273, 8.5, 8.6396144390106198]
Get the pevious arrival time 8.63961443901 
The length of the TD matrix is 7
This is 9.0
This is the iterator 2
This is frow [7.4199999999999999, 7.5596144390106197, 17.25, 18.04285297393799, 19.0, 19.801841862996419, 7.5, 7.949229558308919, 14.5, 15.193816630045573, 18.0, 18.684121640523273, 8.5, 8.6396144390106198, 9.0, 9.0272966245810196]
Get the pevious arrival time 9.02729662458 
The length of the TD matrix is 6
This is 12.0
This is the iterator 3
This is frow [7.4199999999999999, 7.5596144390106197, 17.25, 18.04285297393799, 19.0, 19.801841862996419, 7.5, 7.949229558308919, 14.5, 15.193816630045573, 18.0, 18.684121640523273, 8.5, 8.6396144390106198, 9.0, 9.027296624

In [101]:
results_frow

[8.5,
 8.6396144390106198,
 9.0,
 9.0272966245810196,
 9.5,
 9.5,
 12.0,
 12.157189607620239,
 12.5,
 12.645884958902995,
 17.0,
 17.091132076581321,
 18.0,
 18.095376960436504,
 19.0,
 19.14600617090861]

In [141]:
# r = list(chain(*results_frow))
temp_df2['start_time'] = results_frow[0::2]
temp_df2['arrival_time'] = results_frow[1::2]
temp_df2.iloc[5:19,]

C:\Users\mzd\AppData\Local\Continuum\Anaconda3.6\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\mzd\AppData\Local\Continuum\Anaconda3.6\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,origin_activity,destination_activity,trip_purpose,trip_id,tour_id,tour_seq,is_virtual,hhid,pid,origin_zone,destination_zone,trip_km,primary_mode,tour_starttime,ttime_mins,direction,start_time,arrival_time
362861,work,home,HBW,8442721,7514367,1,False,150574,1,2818,1116,-1.000000,Auto Driver,14.50,41.047298,inbound,18.00,18.684122
362862,home,escort,HBE,24357745,7460046,0,False,150575,1,1116,1198,6.862895,Auto Driver,8.50,8.376866,outbound,8.50,8.639614
362863,escort,other,NHBO,24357746,7460046,1,False,150575,1,1198,1212,1.091865,Auto Driver,8.50,1.637797,inbound,9.00,9.027297
362864,other,escort,NHBO,24357747,7460046,2,False,150575,1,1212,1212,0.436369,Auto Driver,8.50,0.000000,inbound,12.00,12.000000
362865,escort,home,HBE,24357748,7460046,3,False,150575,1,1212,1116,7.681744,Auto Driver,8.50,9.431376,inbound,12.00,12.157190
362866,home,escort,HBE,24357749,7460047,0,False,150575,1,1116,1247,7.081306,Auto Driver,12.50,8.753098,outbound,12.50,12.645885
362867,escort,home,HBE,24357752,7460047,3,False,150575,1,1243,1116,4.421058,Auto Driver,12.50,5.467925,inbound,15.00,15.091132
362868,home,escort,HBE,8376452,7460048,0,False,150575,1,1116,1136,4.643684,Auto Driver,18.00,5.722618,outbound,18.00,18.095377
362869,home,escort,HBE,8376454,7460049,0,False,150575,1,1116,1312,7.135444,Auto Driver,19.00,8.760370,outbound,19.00,19.146006
362870,home,other,HBO,8567834,7604003,0,False,150576,1,1116,1325,12.609245,Auto Driver,8.75,14.223674,outbound,8.75,8.987061


In [151]:
temp_df2['start_time2Check'] = temp_df2['start_time'].shift(-1)
temp_df2['pidCheck'] = temp_df2['pid'].shift(-1)
temp_df2['hidCheck'] = temp_df2['hhid'].shift(-1)
temp_df2['tsCheck'] = temp_df2['tour_seq'].shift(-1)

C:\Users\mzd\AppData\Local\Continuum\Anaconda3.6\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\mzd\AppData\Local\Continuum\Anaconda3.6\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\mzd\AppData\Local\Continuum\Anaconda3.6\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [153]:
temp_df2['checkall'] = temp_df2['start_time2Check']-temp_df2['arrival_time']
temp_df2.loc[(temp_df2['pid'] != temp_df2['pidCheck']) | (temp_df2['hhid'] != temp_df2['hidCheck']) | 
             (temp_df2['tsCheck'] == 0),'checkall'] = 9999

# temp_df2['checkall'] = np.where(temp_df2['start_time2Check']-temp_df2['arrival_time'], 
#                                 (temp_df2['pid'] == temp_df2['pidCheck']) & (temp_df2['hhid'] == temp_df2['hidCheck']))

C:\Users\mzd\AppData\Local\Continuum\Anaconda3.6\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\mzd\AppData\Local\Continuum\Anaconda3.6\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [154]:
temp_df2.loc[temp_df2['checkall']<0]

,origin_activity,destination_activity,trip_purpose,trip_id,tour_id,tour_seq,is_virtual,hhid,pid,origin_zone,...,tour_starttime,ttime_mins,direction,start_time,arrival_time,start_time2Check,pidCheck,hidCheck,checkall,tsCheck
363054,other,shop,NHBO,20160398,7485817,1,False,150670,0,3340,...,9.0,17.272972,inbound,14.0,14.287883,11.92,0.0,150670.0,-2.367883,1.0
363425,escort,home,HBE,8518607,7568541,1,False,150842,0,1220,...,8.0,5.193556,inbound,18.5,18.586559,10.00,0.0,150842.0,-8.586559,1.0
363485,other,home,HBO,8344853,7400400,1,False,150875,0,1067,...,7.0,5.462384,inbound,21.5,21.591040,10.00,0.0,150875.0,-11.591040,1.0
363524,escort,home,HBE,8362641,7426780,1,False,150882,1,1426,...,12.5,16.478453,inbound,15.5,15.774641,15.33,1.0,150882.0,-0.444641,1.0


In [158]:
temp_df2.loc[temp_df2['hhid'] == 150882]

,origin_activity,destination_activity,trip_purpose,trip_id,tour_id,tour_seq,is_virtual,hhid,pid,origin_zone,...,tour_starttime,ttime_mins,direction,start_time,arrival_time,start_time2Check,pidCheck,hidCheck,checkall,tsCheck
363516,other,work,NHBW,24375386,7545330,1,False,150882,0,1088,...,7.83,3.544485,inbound,7.830000,7.889075,12.000000,0.0,150882.0,4.110925,3.0
363517,other,home,HBO,24375388,7545330,3,False,150882,0,1056,...,7.83,3.619817,inbound,12.000000,12.060330,8.500000,1.0,150882.0,9999.000000,0.0
363518,home,escort,HBE,27209485,7426779,0,False,150882,1,1116,...,8.50,8.542444,outbound,8.500000,8.642374,11.500000,1.0,150882.0,2.857626,1.0
363519,escort,shop,NHBO,27209486,7426779,1,False,150882,1,1258,...,8.50,9.934566,inbound,11.500000,11.665576,12.000000,1.0,150882.0,0.334424,2.0
363520,shop,shop,NHBO,27209487,7426779,2,False,150882,1,1384,...,8.50,11.139512,inbound,12.000000,12.185659,12.185659,1.0,150882.0,0.000000,4.0
363521,shop,escort,NHBO,27209489,7426779,4,False,150882,1,1077,...,8.50,1.132769,inbound,12.185659,12.204538,12.204538,1.0,150882.0,0.000000,5.0
363522,escort,home,HBE,27209490,7426779,5,False,150882,1,1061,...,8.50,4.297359,inbound,12.204538,12.276161,12.500000,1.0,150882.0,9999.000000,0.0
363523,home,escort,HBE,8362640,7426780,0,False,150882,1,1116,...,12.50,17.042614,outbound,12.500000,12.784044,15.500000,1.0,150882.0,2.715956,1.0
363524,escort,home,HBE,8362641,7426780,1,False,150882,1,1426,...,12.50,16.478453,inbound,15.500000,15.774641,15.330000,1.0,150882.0,-0.444641,1.0
363525,escort,home,HBE,8362643,7426781,1,False,150882,1,1125,...,15.33,3.936855,inbound,15.330000,15.395614,16.500000,1.0,150882.0,1.104386,2.0


In [26]:
temp_df1.to_csv(r"c:/personal/imm/start_time_algorithm_trial.csv")

In [64]:
%%time
# temp_df1['start_times_clean'] = temp_df1['start_time'].astype('str')
# temp_df1['start_times_clean'] = int(''.join(list(filter(str.isdigit, temp_df1['start_times_clean']))))


Wall time: 0 ns


In [147]:
temp_df2.shape

(896, 22)

## Crazy square brackets

In [89]:
attempt = temp_df1.loc[temp_df1['hhid'] == 150575]
attempt['start_times_clean'].replace(regex=True, inplace=True, to_replace=r'[^0-9.\-]', value=r'')
# attempt['start_times_clean'] = attempt['start_times_clean'].astype(float64)
attempt

C:\Users\mzd\AppData\Local\Continuum\Anaconda3.6\lib\site-packages\pandas\core\generic.py:4619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,origin_activity,destination_activity,trip_purpose,trip_id,tour_id,tour_seq,is_virtual,hhid,pid,origin_zone,destination_zone,trip_km,primary_mode,tour_starttime,ttime_mins,direction,start_time,arrival_time,start_times_clean
362862,home,escort,HBE,24357745,7460046,0,False,150575,1,1116,1198,6.862895,Auto Driver,8.5,8.376866,outbound,8.5,8.63961,8.5
362863,escort,other,NHBO,24357746,7460046,1,False,150575,1,1198,1212,1.091865,Auto Driver,8.5,1.637797,inbound,9,9.0273,9.0
362864,other,escort,NHBO,24357747,7460046,2,False,150575,1,1212,1212,0.436369,Auto Driver,8.5,0.000000,inbound,11.5,11.5,11.5
362865,escort,home,HBE,24357748,7460046,3,False,150575,1,1212,1116,7.681744,Auto Driver,8.5,9.431376,inbound,[11.5],[11.6571896076],11.5
362866,home,escort,HBE,24357749,7460047,0,False,150575,1,1116,1247,7.081306,Auto Driver,12.5,8.753098,outbound,12.5,12.6459,12.5
362867,escort,home,HBE,24357752,7460047,3,False,150575,1,1243,1116,4.421058,Auto Driver,12.5,5.467925,inbound,9.5,9.59113,9.5
362868,home,escort,HBE,8376452,7460048,0,False,150575,1,1116,1136,4.643684,Auto Driver,18.0,5.722618,outbound,18,18.0954,18.0
362869,home,escort,HBE,8376454,7460049,0,False,150575,1,1116,1312,7.135444,Auto Driver,19.0,8.760370,outbound,19,19.146,19.0


In [ ]:
%%time
from itertools import chain
# flatten the nested lists and then get ready for concantenating the results to the master dataframe
l = list(chain(*results1))
l_df = pd.DataFrame(l, index = temp_df1.index, columns=['start_time', 'arrival_time'])

# attach the times
temp_df1 = pd.concat([temp_df1, l_df], axis = 1)

In [ ]:
temp_df1.head(10)

In [ ]:
temp_df1.sort_values(['hhid', 'pid', 'start_time'])

In [ ]:
t10 = temp_df1['start_time'].value_counts().reset_index()
t10.sort_values('index')

In [ ]:
trips_above930 = t10.loc[t10['start_time']>= 21.5]
trips_above930.head()

In [ ]:
t11 = temp_df1['tour_starttime'].value_counts().reset_index()
t11.sort_values('index')

In [ ]:
for _ in range(5):
    print(np.random.RandomState(12345).uniform())

In [ ]:
rando = np.random.RandomState(12345)
for _ in range(5):
    print(rando.uniform())

In [ ]:
def my_function(seed=1235):
    print("BEGIN")
    randomizer = np.random.RandomState(seed)
    for _ in range(5):
        print(randomizer.uniform())

In [ ]:
my_function()
my_function()